In [ ]:
ls ./data

There are about 50 JSON files in the directory. Let's see what each JSON file contains.

In [ ]:
cat data/file_0.json

In [ ]:
from pyspark.sql.types import *

inputPath = "./data/"

# Since we know the data format already, let's define the schema to speed up processing (no need for Spark to infer schema)
jsonSchema = StructType([ StructField("last_reported", TimestampType(), True), 
                          StructField("name", StringType(), True),
                          StructField("station_id", StringType(), True),
                          StructField("region_id", StringType(), True),
                          StructField("publisher", StringType(), True),
                          StructField("lat", FloatType(), True),
                          StructField("lon", FloatType(), True),
                          StructField("country_code", StringType(), True),
                          StructField("num_bikes_available", IntegerType(), True),
                          StructField("num_docks_available", IntegerType(), True),
                          StructField("is_renting", IntegerType(), True),
                          StructField("is_returning", IntegerType(), True)
                        ])


## Static Streaming

In [ ]:
# Static DataFrame representing data in the JSON files
staticInputDF = (
  spark
    .read
    .schema(jsonSchema)
    .json(inputPath)
)

#display(staticInputDF)
staticInputDF.count()

Now we can compute the number of "open" and "close" actions with one hour windows. To do this, we will group by the `action` column and 1 hour windows over the `time` column.

In [ ]:
from pyspark.sql.functions import *      # for window() function

staticCountsperCountryDF = (
  staticInputDF
    .groupBy(
       staticInputDF.country_code, 
       window(staticInputDF.last_reported, "1 minute"))    
    .count()
)
staticCountsperCountryDF.cache()

# Register the DataFrame as table 'static_counts'
staticCountsperCountryDF.createOrReplaceTempView("country_counts")

In [ ]:
spark.sql("select country_code, sum(count) as total_count from country_counts group by country_code").show()

In [ ]:
spark.sql("select * from country_counts where country_code = 'AU'").show()

How about a timeline of windowed counts?

In [ ]:
spark.sql("select country_code, date_format(window.end, 'MMM-dd HH:mm') as time, count from country_counts order by time, country_code").show()

Note the two ends of the graph. The close actions are generated such that they are after the corresponding open actions, so there are more "opens" in the beginning and more "closes" in the end.

## Stream Processing 
Now that we have analyzed the data interactively, let's convert this to a streaming query that continuously updates as data comes. Since we just have a static set of files, we are going to emulate a stream from them by reading one file at a time, in the chronological order they were created. The query we have to write is pretty much the same as the interactive query above.

In [ ]:
from pyspark.sql.functions import *

# Similar to definition of staticInputDF above, just using `readStream` instead of `read`
streamingInputDF = (
  spark
    .readStream                       
    .schema(jsonSchema)               # Set the schema of the JSON data
    .option("maxFilesPerTrigger", 1)  # Treat a sequence of files as a stream by picking one file at a time
    .json(inputPath)
)

# Same query as staticInputDF
streamingCountsDF = (                 
  streamingInputDF
    .groupBy(
      streamingInputDF.country_code, 
      window(streamingInputDF.last_reported, "1 minute"))
    .count()
)

# Is this DF actually a streaming DF?
streamingCountsDF.isStreaming

As you can see, `streamingCountsDF` is a streaming Dataframe (`streamingCountsDF.isStreaming` was `true`). You can start streaming computation, by defining the sink and starting it. 
In our case, we want to interactively query the counts (same queries as above), so we will set the complete set of 1 hour counts to be in a in-memory table (note that this for testing purpose only in Spark 2.0).

In [ ]:
spark.conf.set("spark.sql.shuffle.partitions", "2")  # keep the size of shuffles small

query = (
  streamingCountsDF
    .writeStream
    .format("memory")        # memory = store in-memory table (for testing only in Spark 2.0)
    .outputMode("complete")  # complete = all the counts should be in the table
    .queryName("counts")     # counts = name of the in-memory table
    .option("checkpointLocation", "./checkpoints")
    .start()
)

In [ ]:
# write to file in append mode
#query = (
#  streamingCountsDF
#    .writeStream
#    .queryName("counts")
#    .format("parquet") 
#    .outputMode("update")
#    .option("path", "./outdir")
#    .option("checkpointLocation", "./checkpoints")
#    .start()
#)

`query` is a handle to the streaming query that is running in the background. This query is continuously picking up files and updating the windowed counts. 

Note the status of query in the above cell. Both the `Status: ACTIVE` and the progress bar shows that the query is active. 
Furthermore, if you expand the `>Details` above, you will find the number of files they have already processed. 

Let's wait a bit for a few files to be processed and then interactively query the in-memory `counts` table.

In [ ]:
# RUN a simple webservice on default port 5000 to return the in-memory table as is
from flask import Flask
import json
app = Flask(__name__)

# route to access data
@app.route("/")
def count_per_country():
    counts = spark.sql("select country_code, sum(count) as total_count from counts group by country_code").toJSON().collect()
    return json.dumps(counts)

if __name__ == "__main__":
    app.run()

In [ ]:
from time import sleep
sleep(2)  # wait a bit for computation to start

In [ ]:
spark.sql("select country_code, date_format(window.end, 'MMM-dd HH:mm') as time, count from counts order by time, country_code").show()

We see the timeline of windowed counts (similar to the static one ealrier) building up. If we keep running this interactive query repeatedly, we will see the latest updated counts which the streaming query is updating in the background.

In [ ]:
sleep(2)  # wait a bit more for more data to be computed
spark.sql("select country_code, date_format(window.end, 'MMM-dd HH:mm') as time, count from counts order by time, country_code").show()

Also, let's see the total number of "opens" and "closes".

In [ ]:
spark.sql("select country_code, sum(count) as total_count from counts group by country_code order by country_code").collect()

In [ ]:
# check for running streams:
# spark.streams.active

#Terminate the query stream
#query.stop()

In [ ]:
# some stats for the query
# print(query.lastProgress)